<a href="https://colab.research.google.com/github/tunde99/masakhane-mt/blob/master/en_ha_v1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Masakhane - Machine Translation for African Languages (Using JoeyNMT)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you want to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "ha" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline_v1.2" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag


# This will save it to a folder in our gdrive instead! 
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
g_drive_path = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)
os.environ["gdrive_path"] = g_drive_path
models_path = '%s/models/%s%s_transformer'% (g_drive_path, source_language, target_language)
# model temporary directory for training
model_temp_dir = "/content/drive/My Drive/masakhane/model-temp"
# model permanent storage on the drive
!mkdir -p "$gdrive_path/models/${src}${tgt}_transformer/"

In [3]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/en-ha-baseline_v1.2


In [4]:
# Install opus-tools
! pip install opustools-pkg

     |████████████████████████████████| 81kB 6.8MB/s 


In [5]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-ha.xml.gz not found. The following files are available for downloading:

   3 MB https://object.pouta.csc.fi/OPUS-JW300/v1b/xml/en-ha.xml.gz
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1b/xml/en.zip
  24 MB https://object.pouta.csc.fi/OPUS-JW300/v1b/xml/ha.zip

 289 MB Total size
./JW300_latest_xml_en-ha.xml.gz ... 100% of 3 MB
./JW300_latest_xml_en.zip ... 100% of 263 MB
./JW300_latest_xml_ha.zip ... 100% of 24 MB


In [6]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

# Uncomment the following lines of code if your language pair is part of the global test set
# else, create the test set and upload them to this notebook.

# ! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
# ! mv test.en-$trg.en test.en
# ! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
# ! mv test.en-$trg.$trg test.$trg

--2021-05-04 13:43:40--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.01s   

2021-05-04 13:43:40 (21.3 MB/s) - ‘test.en-any.en’ saved [277791/277791]



In [7]:
# Loading created test sets
from google.colab import files
upload_files = files.upload()

Saving test.en to test.en
Saving test.ha to test.ha


In [8]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [9]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 5417/237064 lines since contained in test set.


,source_sentence,target_sentence
0,Questions From Readers,Tambayoyi Daga Masu Karatu
1,How seriously should Christians view an engage...,Yaya ya kamata Kiristoci su ɗauki riƙo ?
2,An engagement to marry is a cause for happines...,"Riƙo dalili ne na farinciki , amma kuma muhimm..."


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [10]:
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np
from os import cpu_count
from functools import partial
from multiprocessing import Pool


# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

     |████████████████████████████████| 51kB 5.2MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149826 sha256=3761057294e52cf6db2df7868cfa021f1ebed26a304c5d15aaec4ca61ec9bb6d
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [12]:
# NOTE - This might run slow depending on the size of your training set. We are
# printing some information to help you track how long it would take. 
scores = []
start_time = time.time()
for idx, row in df_pp.iterrows():
  scores.append(fuzzfilter(row['source_sentence'], list(en_test_sents), 5))
  if idx % 1000 == 0:
    hours, rem = divmod(time.time() - start_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_pp))))

# Filter out "almost overlapping samples"
df_pp['scores'] = scores
df_pp = df_pp[df_pp['scores'] < 95]

00:00:00.07 0.00 percent complete
00:00:19.37 0.48 percent complete
00:00:38.69 0.96 percent complete
00:00:58.35 1.44 percent complete
00:01:18.05 1.91 percent complete
00:01:37.79 2.39 percent complete
00:01:56.71 2.87 percent complete
00:02:16.47 3.35 percent complete
00:02:35.65 3.83 percent complete
00:02:54.80 4.31 percent complete
00:03:13.63 4.78 percent complete
00:03:33.53 5.26 percent complete
00:03:53.02 5.74 percent complete
00:04:12.48 6.22 percent complete
00:04:32.17 6.70 percent complete
00:04:51.82 7.18 percent complete
00:05:12.17 7.65 percent complete
00:05:31.79 8.13 percent complete
00:05:50.47 8.61 percent complete
00:06:09.83 9.09 percent complete
00:06:29.92 9.57 percent complete
00:06:49.34 10.05 percent complete
00:07:09.11 10.52 percent complete
00:07:28.25 11.00 percent complete
00:07:47.40 11.48 percent complete
00:08:06.82 11.96 percent complete
00:08:26.66 12.44 percent complete
00:08:46.15 12.92 percent complete
00:09:05.53 13.39 percent complete
00:09:

00:29:24.51 43.05 percent complete
00:29:44.56 43.53 percent complete
00:30:03.98 44.01 percent complete
00:30:23.24 44.49 percent complete
00:30:42.34 44.96 percent complete
00:31:01.83 45.44 percent complete
00:31:21.77 45.92 percent complete
00:31:41.49 46.40 percent complete
00:32:00.79 46.88 percent complete
00:32:20.16 47.36 percent complete
00:32:40.42 47.83 percent complete
00:33:00.08 48.31 percent complete
00:33:19.53 48.79 percent complete
00:33:39.70 49.27 percent complete
00:33:59.02 49.75 percent complete
00:34:19.33 50.23 percent complete
00:34:38.66 50.70 percent complete
00:34:57.62 51.18 percent complete
00:35:17.75 51.66 percent complete
00:35:37.11 52.14 percent complete
00:35:57.09 52.62 percent complete
00:36:16.86 53.10 percent complete
00:36:35.92 53.57 percent complete
00:36:55.63 54.05 percent complete
00:37:14.92 54.53 percent complete
00:37:35.06 55.01 percent complete
00:37:54.26 55.49 percent complete


00:38:13.76 55.97 percent complete
00:38:33.12 56.45 percent complete
00:38:52.83 56.92 percent complete
00:39:12.89 57.40 percent complete
00:39:32.03 57.88 percent complete
00:39:51.54 58.36 percent complete
00:40:11.00 58.84 percent complete
00:40:30.25 59.32 percent complete
00:40:49.58 59.79 percent complete
00:41:08.49 60.27 percent complete


00:41:28.18 60.75 percent complete
00:41:47.90 61.23 percent complete
00:42:07.11 61.71 percent complete
00:42:27.01 62.19 percent complete
00:42:47.13 62.66 percent complete
00:43:06.88 63.14 percent complete
00:43:27.18 63.62 percent complete
00:43:47.27 64.10 percent complete
00:44:06.69 64.58 percent complete
00:44:25.99 65.06 percent complete
00:44:45.07 65.53 percent complete
00:45:04.56 66.01 percent complete
00:45:23.74 66.49 percent complete
00:45:43.55 66.97 percent complete
00:46:02.95 67.45 percent complete
00:46:22.50 67.93 percent complete
00:46:41.89 68.40 percent complete
00:47:01.30 68.88 percent complete
00:47:20.92 69.36 percent complete
00:47:41.12 69.84 percent complete
00:48:00.99 70.32 percent complete
00:48:20.32 70.80 percent complete
00:48:40.54 71.27 percent complete
00:49:00.71 71.75 percent complete
00:49:20.79 72.23 percent complete
00:49:39.82 72.71 percent complete
00:49:58.96 73.19 percent complete
00:50:17.89 73.67 percent complete
00:50:37.62 74.14 pe

00:53:13.56 77.97 percent complete
00:53:33.08 78.45 percent complete
00:53:52.78 78.93 percent complete
00:54:11.63 79.41 percent complete
00:54:31.07 79.88 percent complete
00:54:50.90 80.36 percent complete
00:55:10.12 80.84 percent complete
00:55:29.51 81.32 percent complete
00:55:49.64 81.80 percent complete
00:56:09.07 82.28 percent complete
00:56:28.71 82.75 percent complete
00:56:48.71 83.23 percent complete
00:57:07.79 83.71 percent complete
00:57:27.17 84.19 percent complete
00:57:47.11 84.67 percent complete
00:58:06.90 85.15 percent complete
00:58:26.27 85.62 percent complete
00:58:46.12 86.10 percent complete
00:59:06.03 86.58 percent complete
00:59:25.82 87.06 percent complete
00:59:45.44 87.54 percent complete
01:00:04.46 88.02 percent complete
01:00:23.61 88.49 percent complete
01:00:43.05 88.97 percent complete
01:01:02.73 89.45 percent complete
01:01:22.51 89.93 percent complete


01:01:42.08 90.41 percent complete
01:02:01.62 90.89 percent complete
01:02:21.33 91.36 percent complete
01:02:40.13 91.84 percent complete
01:02:59.75 92.32 percent complete
01:03:19.42 92.80 percent complete
01:03:39.43 93.28 percent complete
01:03:59.16 93.76 percent complete
01:04:19.12 94.23 percent complete
01:04:38.69 94.71 percent complete
01:04:57.91 95.19 percent complete
01:05:17.16 95.67 percent complete
01:05:36.83 96.15 percent complete
01:05:56.27 96.63 percent complete
01:06:16.20 97.10 percent complete
01:06:35.97 97.58 percent complete
01:06:55.74 98.06 percent complete
01:07:15.25 98.54 percent complete
01:07:34.66 99.02 percent complete
01:07:54.74 99.50 percent complete
01:08:14.97 99.97 percent complete


In [13]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.en <==
“ Preaching in a territory where you meet individuals who are so eager to learn the truth that they want to study the Bible with you every day is such a joy , ” says Stephanie .
Where a public reading was forbidden , some sent copies to every member of the church . They wanted no further dealings with false religion !
How can couples apply Ephesians 4 : 26 , 27 in their marriage ?
Why may we look to the future with confidence ?
However , she did what she could , and this can teach us a lesson .
Consider what happened just before his arrest , trial , and execution .
Many overcame their superstitious fears , and the number attending grew to over 80 .
This spectacular display attracts tourists from all over the country .
Therefore , beloved ones , since you are awaiting these things , do your utmost to be found finally by him spotless and unblemished and in peace . ”
God’s Word , the Bible , foretold that in our time people would be “ self - assuming , haughty . ”

==> tr



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [15]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .
# Install Pytorch with GPU support v1.7.1.
! pip install torch==1.8+cu101 -f https://download.pytorch.org/whl/torch_stable.html

fatal: destination path 'joeynmt' already exists and is not an empty directory.
Processing /content/joeynmt
  Created wheel for joeynmt: filename=joeynmt-1.3-cp37-none-any.whl size=84842 sha256=85ce3b5f498f4c2e75b4b74dcdb0fb140e5bce6e152ae8ffa59a4de8999945c2
  Stored in directory: /tmp/pip-ephem-wheel-cache-e31uj9h_/wheels/db/01/db/751cc9f3e7f6faec127c43644ba250a3ea7ad200594aeda70a
Successfully built joeynmt
  Found existing installation: joeynmt 1.3
    Uninstalling joeynmt-1.3:
      Successfully uninstalled joeynmt-1.3
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 763.5MB 23kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.8.0+cu101 which is incompatible.
  Found existing installation: torch 1.8.0
    Uninstalling torch-1.8.0:
      Successfully uninstalled torch-1.8.0


# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [16]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# Some output
! echo "BPE Hausa Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  # Herman

bpe.codes.4000	dev.en	     test.bpe.ha     test.ha	   train.en
dev.bpe.en	dev.ha	     test.en	     train.bpe.en  train.ha
dev.bpe.ha	test.bpe.en  test.en-any.en  train.bpe.ha
bpe.codes.4000	dev.en	test.bpe.en  test.en-any.en  train.bpe.ha
dev.bpe.en	dev.ha	test.bpe.ha  test.ha	     train.en
dev.bpe.ha	models	test.en      train.bpe.en    train.ha
BPE Hausa Sentences
Kada mu zama kamar Ab@@ ner da Ab@@ sal@@ om da suka ci am@@ ana .
[ 1 ] ( sa@@ kin la@@ yi na 7 ) An canja wasu sun@@ a@@ ye .
Idan muna da aminci da kuma tawali’u , za mu mai da hankali ga abin da Jehobah yake so mu yi , ba abin da muke so ba
Idan mu masu aminci ne da kuma kirki , za mu riƙa taimaka wa mutane , kuma za mu bi da su yadda Jehobah yake so
Idan muna da aminci da kuma ƙarfin zuciya , za mu yi biyayya ga Jehobah ko da yin hakan yana da wuya ko kuma muna far@@ gabaCombined BPE Vocab
ö
ş
⁄
refore
+
Ž@@
Ó@@
£
tabl@@
↓


# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [18]:
def get_last_checkpoint(directory):
  last_checkpoint = ''
  try:
    for filename in os.listdir(directory):
      if not 'best' in filename and filename.endswith(".ckpt"):
          if not last_checkpoint or int(filename.split('.')[0]) > int(last_checkpoint.split('.')[0]):
            last_checkpoint = filename
  except FileNotFoundError as e:
    print('Error Occur ', e)
  return last_checkpoint

In [19]:
# Copy the created models from the temporary storage to main storage on google drive for persistant storage 
# the content of the folder will be overwritten when you start training
!cp -r "/content/drive/My Drive/masakhane/model-temp/"* "$gdrive_path/models/${src}${tgt}_transformer/"
last_checkpoint = get_last_checkpoint(models_path)
print('Last checkpoint :',last_checkpoint)

cp: cannot stat '/content/drive/My Drive/masakhane/model-temp/*': No such file or directory
Last checkpoint : 


In [20]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [21]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

2021-05-04 15:12:34,252 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-05-04 15:12:34,308 - INFO - joeynmt.data - Loading training data...
2021-05-04 15:12:37,640 - INFO - joeynmt.data - Building vocabulary...
2021-05-04 15:12:37,906 - INFO - joeynmt.data - Loading dev data...
2021-05-04 15:12:37,930 - INFO - joeynmt.data - Loading test data...
2021-05-04 15:12:38,562 - INFO - joeynmt.data - Data loaded.
2021-05-04 15:12:38,562 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-05-04 15:12:38,758 - INFO - joeynmt.model - Enc-dec model built.
2021-05-04 15:12:38.915646: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-05-04 15:12:40,518 - INFO - joeynmt.training - Total params: 12151808
2021-05-04 15:12:42,890 - INFO - joeynmt.helpers - cfg.name                           : enha_transformer
2021-05-04 15:12:42,890 - INFO - joeynmt.helpers - cfg.data.src                       : en
2021-0

In [23]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!cp -r joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer/"

In [24]:
# Output our validation accuracy
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

Steps: 1000	Loss: 95550.13281	PPL: 44.59177	bleu: 1.29115	LR: 0.00030000	*
Steps: 2000	Loss: 81565.06250	PPL: 25.57783	bleu: 3.43028	LR: 0.00030000	*
Steps: 3000	Loss: 73370.53906	PPL: 18.46805	bleu: 6.78752	LR: 0.00030000	*
Steps: 4000	Loss: 68123.66406	PPL: 14.99190	bleu: 8.88514	LR: 0.00030000	*
Steps: 5000	Loss: 63590.62500	PPL: 12.52027	bleu: 11.90724	LR: 0.00030000	*
Steps: 6000	Loss: 60572.60938	PPL: 11.10506	bleu: 14.53174	LR: 0.00030000	*
Steps: 7000	Loss: 58054.21094	PPL: 10.04736	bleu: 15.90478	LR: 0.00030000	*
Steps: 8000	Loss: 56168.58594	PPL: 9.32191	bleu: 17.44216	LR: 0.00030000	*
Steps: 9000	Loss: 54043.51562	PPL: 8.56692	bleu: 18.71990	LR: 0.00030000	*
Steps: 10000	Loss: 52418.21094	PPL: 8.03102	bleu: 19.22068	LR: 0.00030000	*
Steps: 11000	Loss: 51041.77344	PPL: 7.60348	bleu: 20.11009	LR: 0.00030000	*
Steps: 12000	Loss: 50055.09375	PPL: 7.31109	bleu: 20.57364	LR: 0.00030000	*
Steps: 13000	Loss: 49093.66406	PPL: 7.03699	bleu: 21.58137	LR: 0.00030000	*
Steps: 14000	Loss:

In [25]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"

2021-05-04 18:02:41,869 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-05-04 18:02:41,870 - INFO - joeynmt.data - Building vocabulary...
2021-05-04 18:02:42,137 - INFO - joeynmt.data - Loading dev data...
2021-05-04 18:02:42,147 - INFO - joeynmt.data - Loading test data...
2021-05-04 18:02:42,173 - INFO - joeynmt.data - Data loaded.
2021-05-04 18:02:42,202 - INFO - joeynmt.prediction - Process device: cuda, n_gpu: 1, batch_size per device: 18000 (with beam_size)
2021-05-04 18:02:44,679 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-05-04 18:02:44,901 - INFO - joeynmt.model - Enc-dec model built.
2021-05-04 18:02:44,969 - INFO - joeynmt.prediction - Decoding on dev set (data/enha/dev.bpe.ha)...
2021-05-04 18:02:59,109 - WARNING - sacrebleu - That's 100 lines that end in a tokenized period ('.')
2021-05-04 18:02:59,109 - WARNING - sacrebleu - It looks like you forgot to detokenize your test data, which may hurt your score.
2021-05-04 18:02:59,110 - WARNIN

# Conclusion

This marks the end of my task - training an english to hausa language baseline model, using joeynmt.

Training completed: Tuesday, May 4, 2021

Training duration: 2 hours 21 minutes on GPU for 30 epochs.




# Acknowledgements

Thank you, Colin Leong, for guiding this task.

Thank you joeynmt team for joeynmt

Thank you Masakhane github team for the starter notebook.


**Author**: Tunde Ajayi